In [1]:
import numpy as np
import pandas as pd
from paternity_index import Paternity_Index_Cal, LR_Index_Cal

def label_related(x):

    if x.individual_1 == '2800M' or x.individual_2 == '2800M' or x.individual_1 == '9948' or x.individual_2 == '9948':
        return 'unrelated/others'

    if x.individual_1 == 'GM' and x.individual_2 == 'F' or x.individual_1 == 'M' and x.individual_2 == 'F' or x.individual_1 == 'GM' and x.individual_2 == 'C':
        return 'unrelated/others'

    if x.individual_1 == 'GM' and x.individual_2 == 'M' or x.individual_1 == 'F' and x.individual_2 == 'C' or x.individual_1 == 'M' and x.individual_2 == 'C':
        return 'paternity'

def comparison_func(x):
    try:
        ce1 = float(x['ce_geno'].split(',')[0])
        ce2 = float(x['ce_geno'].split(',')[1])
        ns1 = float(x['nastra_geno'].split(',')[0])
        ns2 = float(x['nastra_geno'].split(',')[1])
        if ( abs( ns1 -  ce1 ) < 1e-6 and  abs( ns2 -  ce2 ) < 1e-6  ) \
        or ( abs( ns1 -  ce2 ) < 1e-6 and  abs( ns2 -  ce1 ) < 1e-6  ):
            return 'exact-match'
        elif ( abs( ns1 -  ce1 ) < 1 and  abs( ns2 -  ce2 ) < 1  ) \
        or ( abs( ns1 -  ce2 ) < 1 and  abs( ns2 -  ce1 ) < 1  ):
            return 'incomp-match'    
        elif ( abs( ns1 -  ce1 ) < 1e-6 and  abs( ns2 -  ce2 ) >= 1 ) \
        or ( abs( ns1 -  ce1 ) >= 1   and  abs( ns2 -  ce2 )  < 1e-6 ) \
        or ( abs( ns1 -  ce2 ) < 1e-6 and  abs( ns2 -  ce1 ) >= 1 ) \
        or ( abs( ns1 -  ce2 ) >= 1   and  abs( ns2 -  ce1 )  < 1e-6 ):
            return 'one-match'
        elif ( abs( ns1 -  ce1 )  < 1  and  abs( ns2 -  ce2 ) >= 1 ) \
        or ( abs( ns1 -  ce1 ) >= 1  and  abs( ns2 -  ce2 )  < 1 ) \
        or ( abs( ns1 -  ce2 )  < 1  and  abs( ns2 -  ce1 ) >= 1 ) \
        or ( abs( ns1 -  ce2 ) >= 1  and  abs( ns2 -  ce1 )  < 1 ):
            return 'incomp-one-match'
        else:
            return 'mismatch'
    except:
        return 'fail'
    
ce_loci = ["CSF1PO", "D12S391", "D13S317", "D16S539", "D18S51", "D19S433", 
            "D1S1656", "D21S11", "D2S1338", "D2S441", "D3S1358", "D5S818", 
            "D6S1043", "D7S820", "D8S1179", "FGA", "PentaD", "PentaE", 
            "TH01", "TPOX", "vWA"]
PIC = Paternity_Index_Cal('CHN_STR_allele_freqs.csv')

def row_genotype(x, child_name, pater_name):
    child_genotypes = [ float(geno) for geno in x[child_name].split(',') ]
    pater_genotypes = [ float(geno) for geno in x[pater_name].split(',') ]
    return PIC.single_paternity_cal(child_genotypes, pater_genotypes, x.locus)


In [2]:
ce_dat = pd.read_csv('./CE-result/PowerPlex21_MR_add_controls_merged.csv')
ce_dat.head(2)
melted_ce_df = pd.melt(ce_dat, id_vars=['locus'], var_name='individual', value_name='ce_geno')
melted_ce_df.head(2)

,locus,individual,ce_geno
0,D2S441,C,"10,12"
1,TPOX,C,"12,12"


In [3]:
ce_dat

,locus,C,M,F,GM,2800M,9948
0,D2S441,"10,12","10,11","10,12","10,12","10,14","11,12"
1,TPOX,"12,12","11,12","8,12","11,11","11,11","8,9"
2,CSF1PO,"10,13","9,13","10,12","9,10","12,12","10,11"
3,D5S818,"10,12","12,13","9,10","11,13","12,12","11,13"
4,D1S1656,"11,19.3","13,19.3","11,15","13,19.3","12,13","14,17"
5,PentaD,"11,12","8,11","11,12","8,13","12,13","8,12"
6,D6S1043,"18,18","12,18","18,19","12,14","12,20","12,12"
7,D19S433,"13.2,14","14,16","13,13.2","13,16","13,14","13,14"
8,D8S1179,"11,13","11,12","11,13","11,15","14,15","12,13"
9,D18S51,"14,14","14,15","14,16","14,15","16,18","15,18"


In [4]:
nastra_dat = pd.read_csv('/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/niming_family_5h.csv')

melted_nastra_df = pd.melt(nastra_dat[['locus', 'C', 'F', 'M', 'GM', '2800M', '9948']], id_vars=['locus'], var_name='individual', value_name='nastra_geno')
melted_nastra_df.head(2)

merged_geno_df = pd.merge(melted_ce_df, melted_nastra_df, on = ['locus', 'individual'])
merged_geno_df.head(2)

merged_geno_df['comp'] = merged_geno_df.apply(lambda x: comparison_func(x), axis=1)
merged_geno_df.pivot(index='locus', columns='individual', values='comp').reset_index()

individual,locus,2800M,9948,C,F,GM,M
0,CSF1PO,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
1,D12S391,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
2,D13S317,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
3,D16S539,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
4,D18S51,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
5,D19S433,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
6,D1S1656,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
7,D21S11,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
8,D2S1338,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match
9,D2S441,exact-match,exact-match,exact-match,exact-match,exact-match,exact-match


In [5]:
counts_df = []

for i in [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]:

    nastra_dat = pd.read_csv(f'/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/niming_family_{i}h.csv')
    melted_nastra_df = pd.melt(nastra_dat[['locus', 'C', 'F', 'M', 'GM', '2800M', '9948']], id_vars=['locus'], var_name='individual', value_name='nastra_geno')

    merged_geno_df = pd.merge(melted_ce_df, melted_nastra_df, on = ['locus', 'individual'])
    merged_geno_df['comp'] = merged_geno_df.apply(lambda x: comparison_func(x), axis=1)

    value_count_df = merged_geno_df.comp.value_counts().reset_index()
    value_count_df['duration'] = f'{i * 60}min'
    counts_df.append(value_count_df)

counts_df = pd.concat(counts_df, axis = 0)

In [6]:
counts_df

,comp,count,duration
0,fail,85,6.0min
1,exact-match,40,6.0min
2,one-match,1,6.0min
0,exact-match,83,12.0min
1,fail,43,12.0min
0,exact-match,97,18.0min
1,fail,29,18.0min
0,exact-match,114,24.0min
1,fail,12,24.0min
0,exact-match,122,30.0min


In [7]:
counts_df.to_csv('/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/df_7b.csv', index=None)

# paternity testing

In [7]:
throughput = pd.read_csv('../data/case_study_ce_result/bases_durations.txt', sep='\t', header=None)
throughput.columns = ['duration', 'individual', 'bases']
throughput.head(2)

,duration,individual,bases
0,6min,GM,1959858
1,6min,F,2736588


In [8]:
Paternity_Index_df = []
for i in [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]:

    nastra_dat = pd.read_csv(f'/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/niming_family_{i}h.csv')
    dat = nastra_dat[nastra_dat.locus.isin(ce_loci)]
    dat = dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA})

    name_lst = dat.columns[1:]

    heat_map_lst = []

    for ind1, obj1 in enumerate(name_lst):
        for ind2, obj2 in enumerate(name_lst):
            if ind1 <= ind2:
                continue
            heat_map_lst.append([obj1, obj2, dat[['locus', obj1, obj2]].dropna(axis=0).apply(lambda x: row_genotype(x, obj1, obj2), axis=1).prod()])

    heat_map_df = pd.DataFrame(heat_map_lst, columns=['individual_1', 'individual_2', 'Paternity_Index'])
    heat_map_df['relationship'] = heat_map_df.apply(lambda x: label_related(x), axis = 1 )
    heat_map_df['duration']     = f'{i * 60}min'
    Paternity_Index_df.append( heat_map_df )

Paternity_Index_df = pd.concat(Paternity_Index_df, axis=0)
Paternity_Index_df.head()


,individual_1,individual_2,Paternity_Index,relationship,duration
0,F,C,1962.153195,paternity,6.0min
1,M,C,24.803128,paternity,6.0min
2,M,F,0.000000,unrelated/others,6.0min
3,GM,C,0.000000,unrelated/others,6.0min
4,GM,F,0.000000,unrelated/others,6.0min


In [9]:
Paternity_Index_df

,individual_1,individual_2,Paternity_Index,relationship,duration
0,F,C,1962.153195,paternity,6.0min
1,M,C,24.803128,paternity,6.0min
2,M,F,0.000000,unrelated/others,6.0min
3,GM,C,0.000000,unrelated/others,6.0min
4,GM,F,0.000000,unrelated/others,6.0min
...,...,...,...,...,...
10,9948,C,0.000000,unrelated/others,300min
11,9948,F,0.000000,unrelated/others,300min
12,9948,M,0.000000,unrelated/others,300min
13,9948,GM,0.000000,unrelated/others,300min


In [11]:
Paternity_Index_df.to_csv('/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/df_7d.csv', index=None)

In [12]:
nastra_dat = pd.read_csv(f'./niming-family/niming_family_0.3h.csv')
dat = nastra_dat[nastra_dat.locus.isin(ce_loci)]
dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA}, inplace=True)

name_lst = dat.columns[1:]

heat_map_lst = []

for ind1, obj1 in enumerate(name_lst):
    for ind2, obj2 in enumerate(name_lst):
        if ind1 == ind2:
            continue
        heat_map_lst.append([obj1, obj2, dat[['locus', obj1, obj2]].dropna(axis=0).apply(lambda x: row_genotype(x, obj1, obj2), axis=1).prod()])

heat_map_df = pd.DataFrame(heat_map_lst, columns=['individual_1', 'individual_2', 'Paternity_Index'])
heat_map_df.pivot(columns="individual_1", index="individual_2", values = 'Paternity_Index')

/var/folders/bb/sy_px5kn2hn7wws66szjnnp80000gn/T/ipykernel_7525/84300311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA}, inplace=True)


individual_1,2800M,9948,C,F,GM,M
individual_2,,,,,,
2800M,NaN,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000
9948,0.0,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000
C,0.0,0.0,NaN,3.097634e+07,0.000000,41823.595733
F,0.0,0.0,3.097634e+07,NaN,0.000000,0.000000
GM,0.0,0.0,0.000000e+00,0.000000e+00,NaN,101652.922014
M,0.0,0.0,4.182360e+04,0.000000e+00,101652.922014,NaN


# individual idetification

In [10]:
LIC = LR_Index_Cal('CHN_STR_allele_freqs.csv')

nastra_dat = pd.read_csv(f'./niming-family/niming_family_0.3h.csv')
dat = nastra_dat[nastra_dat.locus.isin(ce_loci)]
dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA}, inplace=True)
dat.head()

/var/folders/bb/sy_px5kn2hn7wws66szjnnp80000gn/T/ipykernel_13430/972129959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA}, inplace=True)


,locus,C,F,M,GM,2800M,9948
0,CSF1PO,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,D12S391,"17.0,24.0","17.0,24.0","17.0,20.0","17.0,19.0","18.0,23.0","18.0,24.0"
3,D13S317,"11.0,11.0","8.0,11.0","11.0,8.0","8.0,9.0","9.0,11.0","11.0,11.0"
4,D16S539,"11.0,12.0","11.0,13.0","12.0,10.0","12.0,10.0","13.0,9.0","11.0,11.0"
6,D18S51,"14.0,14.0",<NA>,"14.0,15.0",<NA>,<NA>,<NA>


In [11]:
dat.C.count()

18

In [12]:
output = []
for duration in [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]:
    nastra_dat = pd.read_csv(f'/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/niming_family_{duration}h.csv')
    dat = nastra_dat[nastra_dat.locus.isin(ce_loci)]
    dat = dat.replace({'Fail: NoCalling': pd.NA, 'Fail: Interpretation': pd.NA, 'Fail: Imbalance': pd.NA})
    for individual in dat.columns[1:]:
        loci_count = dat[individual].count()
        
        output.append([individual, f'{int(duration * 60)}min', 1/dat.apply(lambda x: LIC.individual_identification_cal(x[individual], x['locus']), axis=1).prod(), loci_count])

output = pd.DataFrame(output, columns=['individual', 'duration', 'LR', 'loci_count'])


In [14]:
output.to_csv('/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/lrs.csv', index=None)

In [17]:
throughput = pd.read_csv('../data/case_study_ce_result/bases_durations.txt', sep='\t', header=None)
throughput.columns = ['duration', 'individual', 'bases']
throughput.head(2)

,duration,individual,bases
0,6min,GM,1959858
1,6min,F,2736588


In [18]:
merged_output = pd.merge(output, throughput, on = ['duration', 'individual'])
merged_output

,individual,duration,LR,loci_count,bases
0,C,6min,2.789020e+15,11,2346159
1,F,6min,3.877567e+10,7,2736588
2,M,6min,2.717148e+10,7,1842980
3,GM,6min,4.689466e+07,6,1959858
4,2800M,6min,2.224571e+03,2,1904737
5,9948,6min,1.461111e+11,8,2521167
6,C,12min,1.792877e+24,16,5142908
7,F,12min,4.030844e+19,14,6123419
8,M,12min,1.352526e+15,12,4018134
9,GM,12min,5.004372e+18,14,4430006


In [19]:
merged_output.to_csv('/Users/renz/Documents/SynologyDrive/1. 研究项目/2024年/202406-NanoSTR_BIB_version/20240725-second-response/figures/fig7/df_7c.csv', index=None)